## 思路一：人工统计特征
在文本分类和自然语言处理中，常见的文本统计特征是指通过对文本数据的统计信息进行提取，将文本转换为数值表示的特征。这些统计特征通常用于构建文本分类模型或其他文本相关任务的输入。以下是一些常见的文本统计特征：
1. **文本长度**： 文本长度是指文本包含的词或字符的数量。文本长度可以在一定程度上反映文本的复杂度和信息量。
2. **句子数量**： 句子数量表示文本中包含的句子数目。句子数量可能与文本的复杂性和组织结构有关。
3. **特殊字符数量**： 特殊字符数量表示文本中包含的特殊字符（如标点符号、数字等）的个数。特殊字符的数量可能与文本的风格和语言表达有关。
4. **词汇丰富性**： 词汇丰富性是指文本中不同词汇的种类数。丰富多样的词汇可能表示文本更加丰富和多样化。
5. **句子平均长度**： 句子平均长度是指文本中句子的平均词数。较长的句子可能包含更多信息。
6. **词汇覆盖率**： 词汇覆盖率表示文本中不同词汇与整个文本数据集中的不同词汇之间的比例。词汇覆盖率高表示文本包含的词汇较为丰富。
7. **情感词频**： 在情感分析任务中，可以统计文本中情感词汇（如积极、消极的词汇）出现的频率，用于判断文本的情感倾向。

方法优缺点（运行时间2分钟）：
- 思路简单，可以人工添加特征
- 精度较差，无法捕获高阶统计特征

In [1]:
import glob # 导入glob模块，用于查找符合特定规则的文件路径名
import numpy as np # 导入NumPy库，用于数值计算
import pandas as pd # 导入Pandas库，用于数据分析和操作
from sklearn.linear_model import LogisticRegression # 导入逻辑回归模型
# LogisticRegression原理: https://juejin.cn/post/7192114678812639293

# 读取训练和测试数据
train_data = pd.read_csv('./data/ChatGPT生成文本检测器公开数据-更新/train.csv') # 读取训练数据
test_data = pd.read_csv('./data/ChatGPT生成文本检测器公开数据-更新/test.csv') # 读取测试数据

# 移除内容列中的第一个和最后一个字符
train_data['content'] = train_data['content'].apply(lambda x: x[1:-1]) # 移除训练数据中内容列的每个字符串的第一个和最后一个字符
test_data['content'] = test_data['content'].apply(lambda x: x[1:-1]) # 移除测试数据中内容列的每个字符串的第一个和最后一个字符

def simple_feature(s):
    if len(s) == 0:
        s = '123 123' # 如果字符串为空，则设为'123 123'
    
    w = s.split() # 这行代码将字符串s按照空格分割，得到一个单词列表w。
    
    # 统计字符出现次数
    w_count = np.bincount(w)

    # 这行代码过滤掉w_count中的0，即过滤掉没有出现过的单词。
    w_count = w_count[w_count != 0]

    return np.array([
        len(s), # 原始字符长度
        len(w), # 字符个数
        len(set(w)), # 不重复字符个数
        len(w) - len(set(w)), # 字符个数 - 不重复字符个数
        len(set(w)) / (len(w) + 1), # 不重复字符个数占比
        np.max(w_count), # 字符的频率的最大值
        np.min(w_count), # 字符的频率的最小值
        np.mean(w_count), # 字符的频率的平均值
        np.std(w_count), # 字符的频率的标准差
        np.ptp(w_count), # 字符的频率的极差
    ])

# 对训练和测试数据的内容列应用上述特征提取函数
train_feature = train_data['content'].iloc[:].apply(simple_feature) # 对训练数据的内容列应用特征提取函数
test_feature = test_data['content'].iloc[:].apply(simple_feature) # 对测试数据的内容列应用特征提取函数

# 将特征堆叠成NumPy数组(其实形状没有改变)
train_feature = np.vstack(train_feature.values) # 将训练数据的特征堆叠成NumPy数组
test_feature = np.vstack(test_feature.values) # 将测试数据的特征堆叠成NumPy数组
m = LogisticRegression() # 初始化逻辑回归模型
m.fit(train_feature, train_data['label']) # 使用训练数据拟合模型
test_data['label'] = m.predict(test_feature) # 对测试数据进行预测
test_data[['name', 'label']].to_csv('result/simple.csv', index=None) # 将预测结果保存到CSV文件

## 思路二：TFIDF统计特征
TF-IDF（Term Frequency-Inverse Document Frequency）是一种常用的文本特征表示方法，用于衡量一个词在文本中的重要性。TF-IDF结合了词频（TF）和逆文档频率（IDF），用于对每个词赋予一个权重，从而将文本数据转换为数值形式，便于在机器学习算法中使用。

下面对TF和IDF进行简单的解释：
1. **词频**（Term Frequency, TF）： 词频指的是某个词在文本中出现的频率。TF可以通过以下公式计算：
   TF(word, document) = (词word在文档document中出现的次数) / (文档document中的总词数)
   词频衡量了一个词在文本中的重要程度，词频越高表示该词越重要。

2. **逆文档频率**（Inverse Document Frequency, IDF）： IDF用于衡量一个词的稀有程度。它通过对整个文本数据集中包含该词的文档数量进行倒数处理，再取对数得到的值。IDF可以通过以下公式计算：
   IDF(word) = log(文本数据集中的总文档数 / (包含词word的文档数 + 1))
   IDF的值高表示该词较为稀有，可能具有更好的区分性。

3. **TF-IDF**： TF-IDF是TF和IDF的乘积，用于综合考虑一个词在文本中的频率和在整个文本数据集中的稀有程度。TF-IDF可以通过以下公式计算：
   TF-IDF(word, document) = TF(word, document) * IDF(word)
   TF-IDF将词频和逆文档频率结合起来，能够突出每个词对文本的重要性。对于一个给定的文本，TF-IDF将计算出每个词的TF-IDF值，从而形成一个向量表示该文本。
TF-IDF在文本分类、信息检索、搜索引擎以及文本挖掘等任务中广泛应用。它能够有效地对文本进行特征表示，使得模型能够更好地捕捉文本的重要信息，从而提高文本分类和相关任务的性能。

TfidfVectorizer是scikit-learn库中用于将文本数据转换为TF-IDF特征表示的工具。它是一个方便而强大的文本特征提取器。下面介绍一些常用的TfidfVectorizer中的参数：
1. **stop_words**（默认为None）：停用词列表。停用词是那些在文本处理中经常被过滤掉的常见词汇，例如"a"、"the"、"and"等。可以传入一个列表，其中包含要过滤的停用词，TfidfVectorizer会自动在处理文本时忽略这些词汇。
2. **max_df**（默认为1.0）：用于设置词频的阈值。表示忽略在文档中出现频率超过max_df的词汇。可以是绝对的词频计数，也可以是相对的词频比例（例如0.8表示忽略在80%以上的文档中出现的词汇）。
3. **min_df**（默认为1）：用于设置词频的下限。表示忽略在文档中出现频率低于min_df的词汇。可以是绝对的词频计数，也可以是相对的词频比例。
4. **max_features**（默认为None）：指定最大特征数。根据TF-IDF值对所有特征进行排序，选择TF-IDF值最大的前max_features个特征。
5. **ngram_range**（默认为(1, 1)）：指定要考虑的n-gram范围。n-gram是连续的n个词组成的序列。例如，ngram_range=(1, 2)表示同时考虑单个词和相邻的两个词的组合。
6. **norm**（默认为'l2'）：用于对特征向量进行归一化的方式。'l2'表示欧几里得范数（即将向量缩放到单位范数），'l1'表示曼哈顿范数，None表示不归一化。
7. **use_idf**（默认为True）：是否使用逆文档频率（IDF）。如果为True，则计算TF-IDF特征；如果为False，则只计算TF特征。
8. **smooth_idf**（默认为True）：是否平滑逆文档频率（IDF）。如果为True，则在计算IDF时避免除以0，避免出现无穷大值。
9. **sublinear_tf**（默认为False）：是否使用子线性TF缩放。如果为True，则使用1 + log(TF)来替代普通的TF计算。

In [4]:
# 思路二完整代码如下:

import glob # 导入glob模块，用于查找符合特定规则的文件路径名
import numpy as np # 导入NumPy库，用于数值计算
import pandas as pd # 导入Pandas库，用于数据分析和操作
from sklearn.linear_model import LogisticRegression # 导入逻辑回归模型
from sklearn.feature_extraction.text import TfidfVectorizer # 导入TF-IDF向量化器，用于将文本数据转换为数值特征
from sklearn.model_selection import cross_val_predict # 导入交叉验证预测函数
from sklearn.metrics import classification_report # 导入分类报告函数，用于评估模型性能

# 读取训练和测试数据
# train_data = pd.read_csv('./ChatGPT生成文本检测器公开数据-更新/train.csv') # 读取训练数据
# test_data = pd.read_csv('./ChatGPT生成文本检测器公开数据-更新/test.csv') # 读取测试数据

# 移除内容列中的第一个和最后一个字符
train_data['content'] = train_data['content'].apply(lambda x: x[1:-1]) # 移除训练数据中内容列的每个字符串的第一个和最后一个字符
test_data['content'] = test_data['content'].apply(lambda x: x[1:-1]) # 移除测试数据中内容列的每个字符串的第一个和最后一个字符

# 使用第1种TF-IDF参数进行特征提取
tfidf = TfidfVectorizer(token_pattern=r'\w{1}', max_features=2000) # 初始化TF-IDF向量化器，设置单词的最小长度为1，最大特征数量为2000
train_tfidf = tfidf.fit_transform(train_data['content']) # 将训练数据的内容列转换为TF-IDF特征
test_tfidf = tfidf.transform(test_data['content']) # 将测试数据的内容列转换为TF-IDF特征

# 使用交叉验证进行预测，并打印分类报告
print(classification_report(
    cross_val_predict(
        LogisticRegression(), # 使用逻辑回归模型
        train_tfidf, # 训练数据的特征
        train_data['label'], # 训练数据的标签
    ),
    train_data['label'], # 真实的训练数据标签
    digits=4 # 设置打印的小数位数为4
))

# 使用第2种TF-IDF参数进行特征提取
tfidf = TfidfVectorizer(token_pattern=r'\w{1}', max_features=5000) # 初始化TF-IDF向量化器，设置单词的最小长度为1，最大特征数量为5000
train_tfidf = tfidf.fit_transform(train_data['content']) # 将训练数据的内容列转换为TF-IDF特征
test_tfidf = tfidf.transform(test_data['content']) # 将测试数据的内容列转换为TF-IDF特征

# 使用交叉验证进行预测，并打印分类报告
print(classification_report(
    cross_val_predict(
        LogisticRegression(), # 使用逻辑回归模型
        train_tfidf, # 训练数据的特征
        train_data['label'], # 训练数据的标签
    ),
    train_data['label'], # 真实的训练数据标签
    digits=4 # 设置打印的小数位数为4
))

# 使用第3种TF-IDF参数进行特征提取
tfidf = TfidfVectorizer(token_pattern=r'\w{1}', max_features=5000, ngram_range=(1,2)) # 初始化TF-IDF向量化器，设置单词的最小长度为1，最大特征数量为5000，n-gram范围为1到2
train_tfidf = tfidf.fit_transform(train_data['content']) # 将训练数据的内容列转换为TF-IDF特征
test_tfidf = tfidf.transform(test_data['content']) # 将测试数据的内容列转换为TF-IDF特征

# 使用交叉验证进行预测，并打印分类报告
print(classification_report(
    cross_val_predict(
        LogisticRegression(), # 使用逻辑回归模型
        train_tfidf, # 训练数据的特征
        train_data['label'], # 训练数据的标签
    ),
    train_data['label'], # 真实的训练数据标签
    digits=4 # 设置打印的小数位数为4
))

# 使用第3种TF-IDF参数训练逻辑回归模型，并进行预测
m = LogisticRegression() # 初始化逻辑回归模型
m.fit(
    train_tfidf, # 训练数据的特征
    train_data['label'] # 训练数据的标签
)
test_data['label'] = m.predict(test_tfidf) # 对测试数据进行预测

# 将预测结果保存到CSV文件
test_data[['name', 'label']].to_csv('result/tfidf.csv', index=None) # 将测试数据的名称和预测标签保存到CSV文件


              precision    recall  f1-score   support

           0     0.9921    0.8863    0.9362     13248
           1     0.3041    0.8750    0.4513       752

    accuracy                         0.8857     14000
   macro avg     0.6481    0.8807    0.6938     14000
weighted avg     0.9551    0.8857    0.9102     14000
              precision    recall  f1-score   support

           0     0.9921    0.8863    0.9362     13248
           1     0.3041    0.8750    0.4513       752

    accuracy                         0.8857     14000
   macro avg     0.6481    0.8807    0.6938     14000
weighted avg     0.9551    0.8857    0.9102     14000
              precision    recall  f1-score   support

           0     0.9946    0.9074    0.9490     12974
           1     0.4445    0.9376    0.6031      1026

    accuracy                         0.9096     14000
   macro avg     0.7196    0.9225    0.7761     14000
weighted avg     0.9543    0.9096    0.9236     14000


In [7]:
# 使用随机森林模型进行预测
from sklearn.ensemble import RandomForestClassifier # 导入随机森林模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(train_tfidf, train_data['label'])
test_data['label'] = clf.predict(test_tfidf)
test_data[['name', 'label']].to_csv('result/tfidf_rf.csv', index=None) # 将测试数据的名称和预测标签保存到CSV文件

In [8]:
# 使用XGBoost模型进行预测
from xgboost import XGBClassifier # 导入XGBoost模型
clf_xgb = XGBClassifier(n_estimators=100, random_state=42)
clf_xgb.fit(train_tfidf, train_data['label'])
test_data['label'] = clf_xgb.predict(test_tfidf)
test_data[['name', 'label']].to_csv('result/tfidf_xgb.csv', index=None) # 将测试数据的名称和预测标签保存到CSV文件

In [9]:
# 使用SVM模型进行预测
from sklearn.svm import SVC # 导入SVM模型
clf_svm = SVC(C=100, gamma=0.001, kernel='rbf', random_state=42)
clf_svm.fit(train_tfidf, train_data['label'])
test_data['label'] = clf_svm.predict(test_tfidf)
test_data[['name', 'label']].to_csv('result/tfidf_svm.csv', index=None) # 将测试数据的名称和预测标签保存到CSV文件